### Restart and run all cells

#### csv files: sell-alert

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 22)

In [2]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[10]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = trunc(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [3]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}','css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [4]:
sql = '''
SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '%s'
'''
# AND to_price >= fm_price
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '2022-02-22'



,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status
18,DOHOME,24.40,0,"4,000","12,000",24.30,2022-02-21,2022-02-22,1,24.00,23.50,-5,24.00,30.75,27.63,"97,600.00",I
14,GLOBAL,21.80,0,"5,000","10,000",19.40,2022-02-21,2022-02-22,1,21.20,21.10,-1,21.30,24.70,25.75,"109,000.00",I
13,IP,21.20,0,"2,500","7,500",20.20,2022-02-21,2022-02-22,1,20.50,20.40,-1,20.60,24.47,22.20,"53,000.00",I
7,IVL,48.25,12,"1,200","3,600",42.00,2022-02-21,2022-02-22,1,46.00,47.25,5,47.50,52.00,58.49,"57,900.00",I
0,LPF,14.10,0,"10,000","10,000",14.00,2022-02-14,2022-02-22,6,13.00,13.80,8,13.80,17.10,16.50,"141,000.00",I
11,MCS,16.80,3,"5,000","50,000",16.70,2022-02-21,2022-02-22,1,14.60,14.70,1,14.70,16.20,21.00,"84,000.00",I
1,NER,7.80,0,"9,000","27,000",7.45,2022-02-14,2022-02-22,6,7.15,7.50,7,7.65,8.65,10.10,"70,200.00",I
8,PTT,41.25,0,"3,000","9,000",38.50,2022-02-21,2022-02-22,1,38.75,39.00,1,39.25,42.50,48.76,"123,750.00",I
2,RATCH,48.00,3,"3,000","6,000",45.75,2022-02-17,2022-02-22,3,46.50,45.75,-3,46.00,53.75,46.00,"144,000.00",I
3,RCL,46.75,0,"1,500","4,500",44.50,2022-02-14,2022-02-22,6,41.25,44.50,13,44.75,67.75,41.75,"70,125.00",I


In [5]:
my_array = df_stock.to_numpy()
my_array

array([['LPF', 14.1, 0, 10000, 10000, 14.0, '2022-02-14', '2022-02-22',
        6, 13.0, 13.8, 8, 13.8, 17.1, 16.5, 141000.0, 'I'],
       ['NER', 7.8, 0, 9000, 27000, 7.45, '2022-02-14', '2022-02-22', 6,
        7.15, 7.5, 7, 7.65, 8.65, 10.1, 70200.0, 'I'],
       ['RATCH', 48.0, 3, 3000, 6000, 45.75, '2022-02-17', '2022-02-22',
        3, 46.5, 45.75, -3, 46.0, 53.75, 46.0, 144000.0, 'I'],
       ['RCL', 46.75, 0, 1500, 4500, 44.5, '2022-02-14', '2022-02-22', 6,
        41.25, 44.5, 13, 44.75, 67.75, 41.75, 70125.0, 'I'],
       ['STA', 31.5, 0, 2500, 20000, 34.75, '2022-02-17', '2022-02-22',
        3, 32.0, 29.25, -11, 29.75, 56.75, 34.1, 78750.0, 'S'],
       ['TSE', 2.74, 0, 40000, 40000, 2.6, '2022-02-11', '2022-02-22', 7,
        2.48, 2.6, 6, 2.84, 3.7, 3.43, 109600.00000000001, 'I'],
       ['TU', 21.8, 12, 6000, 18000, 20.8, '2022-02-18', '2022-02-22', 2,
        21.1, 20.7, -4, 20.8, 23.0, 25.75, 130800.0, 'I'],
       ['IVL', 48.25, 12, 1200, 3600, 42.0, '2022-02-21', '20

In [6]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([ 2,  6,  9,  9,  9,  7, 11,  4,  9,  5,  7, 21, 14,  8,  6, 11,  9,
       12,  9])

In [7]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status,spd
0,LPF,14.10,0,"10,000","10,000",14.00,2022-02-14,2022-02-22,6,13.00,13.80,8,13.80,17.10,16.50,"141,000.00",I,2
1,NER,7.80,0,"9,000","27,000",7.45,2022-02-14,2022-02-22,6,7.15,7.50,7,7.65,8.65,10.10,"70,200.00",I,6
2,RATCH,48.00,3,"3,000","6,000",45.75,2022-02-17,2022-02-22,3,46.50,45.75,-3,46.00,53.75,46.00,"144,000.00",I,9
3,RCL,46.75,0,"1,500","4,500",44.50,2022-02-14,2022-02-22,6,41.25,44.50,13,44.75,67.75,41.75,"70,125.00",I,9
4,STA,31.50,0,"2,500","20,000",34.75,2022-02-17,2022-02-22,3,32.00,29.25,-11,29.75,56.75,34.10,"78,750.00",S,9
5,TSE,2.74,0,"40,000","40,000",2.60,2022-02-11,2022-02-22,7,2.48,2.60,6,2.84,3.70,3.43,"109,600.00",I,7
6,TU,21.80,12,"6,000","18,000",20.80,2022-02-18,2022-02-22,2,21.10,20.70,-4,20.80,23.00,25.75,"130,800.00",I,11
7,IVL,48.25,12,"1,200","3,600",42.00,2022-02-21,2022-02-22,1,46.00,47.25,5,47.50,52.00,58.49,"57,900.00",I,4
8,PTT,41.25,0,"3,000","9,000",38.50,2022-02-21,2022-02-22,1,38.75,39.00,1,39.25,42.50,48.76,"123,750.00",I,9
9,WHART,12.80,0,"10,000","10,000",12.50,2022-02-21,2022-02-22,1,12.20,12.30,1,12.40,14.08,14.90,"128,000.00",I,5


In [8]:
sql = '''
SELECT name, volbuy AS shares, price AS unit_cost, volbuy * price AS cost_amt, dividend,
volbuy * dividend AS div_amt, (volbuy * dividend)/(volbuy * price) * 100 AS cost_pct, period
FROM buy
WHERE period IN ("1","2","3","4")
'''
df_buy = pd.read_sql(sql, const)
df_buy['shares'] = df_buy.shares.astype(int)
df_buy.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
9,BCH,"15,000",21.30,"319,500.00",0.33,"4,950.00",1.55,3
26,BGRIM,"9,000",40.00,"360,000.00",0.45,"4,050.00",1.12,4
24,CPNCG,"10,000",12.50,"125,000.00",0.96,"9,587.00",7.67,2
5,DIF,"50,000",14.70,"735,000.00",1.04,"52,200.00",7.10,2
11,DOHOME,"12,000",24.30,"291,600.00",0.10,"1,200.00",0.41,4
7,GLOBAL,"10,000",19.40,"194,000.00",nan,nan,nan,4
16,HREIT,"30,000",8.95,"268,500.00",0.70,"20,856.00",7.77,2
31,IP,"7,500",20.20,"151,500.00",0.18,"1,387.50",0.92,3
2,IVL,"3,600",42.00,"151,200.00",0.85,"3,060.00",2.02,3
12,JASIF,"100,000",10.00,"1,000,000.00",0.95,"95,000.00",9.50,2


In [9]:
df_merge = pd.merge(df_stock, df_buy, on=['name'], how='inner')
cols = 'name spd target sh qty available_qty cost fm_price to_price ttl max max52 css_price status period amount'.split()
df_merge[cols].style.format(format_dict)

,name,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
0,LPF,2,14.10,0,"10,000","10,000",14.00,13.00,13.80,8,13.80,17.10,16.50,I,4,"141,000.00"
1,NER,6,7.80,0,"9,000","27,000",7.45,7.15,7.50,7,7.65,8.65,10.10,I,3,"70,200.00"
2,RATCH,9,48.00,3,"3,000","6,000",45.75,46.50,45.75,-3,46.00,53.75,46.00,I,4,"144,000.00"
3,RCL,9,46.75,0,"1,500","4,500",44.50,41.25,44.50,13,44.75,67.75,41.75,I,3,"70,125.00"
4,STA,9,31.50,0,"2,500","20,000",34.75,32.00,29.25,-11,29.75,56.75,34.10,S,1,"78,750.00"
5,TSE,7,2.74,0,"40,000","40,000",2.60,2.48,2.60,6,2.84,3.70,3.43,I,3,"109,600.00"
6,TU,11,21.80,12,"6,000","18,000",20.80,21.10,20.70,-4,20.80,23.00,25.75,I,3,"130,800.00"
7,IVL,4,48.25,12,"1,200","3,600",42.00,46.00,47.25,5,47.50,52.00,58.49,I,3,"57,900.00"
8,PTT,9,41.25,0,"3,000","9,000",38.50,38.75,39.00,1,39.25,42.50,48.76,I,4,"123,750.00"
9,WHART,5,12.80,0,"10,000","10,000",12.50,12.20,12.30,1,12.40,14.08,14.90,I,4,"128,000.00"


In [10]:
sell_alert = df_merge[cols]
sell_alert.set_index(['name'],inplace=True)
sell_alert.sort_values(['spd'],ascending=[True])

,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
name,,,,,,,,,,,,,,,
LPF,2,14.10,0,10000,10000,14.00,13.00,13.80,8,13.80,17.10,16.50,I,4,141000.0
IVL,4,48.25,12,1200,3600,42.00,46.00,47.25,5,47.50,52.00,58.49,I,3,57900.0
WHART,5,12.80,0,10000,10000,12.50,12.20,12.30,1,12.40,14.08,14.90,I,4,128000.0
NER,6,7.80,0,9000,27000,7.45,7.15,7.50,7,7.65,8.65,10.10,I,3,70200.0
GLOBAL,6,21.80,0,5000,10000,19.40,21.20,21.10,-1,21.30,24.70,25.75,I,4,109000.0
TSE,7,2.74,0,40000,40000,2.60,2.48,2.60,6,2.84,3.70,3.43,I,3,109600.0
RJH,7,34.75,0,1500,4500,33.00,32.25,33.00,3,33.25,39.00,37.00,I,4,52125.0
IP,8,21.20,0,2500,7500,20.20,20.50,20.40,-1,20.60,24.47,22.20,I,3,53000.0
RCL,9,46.75,0,1500,4500,44.50,41.25,44.50,13,44.75,67.75,41.75,I,3,70125.0


In [11]:
file_name = 'sell-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

sell_alert.sort_values(['spd'],ascending=[True]).to_csv(output_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(data_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(box_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(one_file)